<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/Emails_Processing_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Email Analysis Notebook

# Email Parsing & Initial Processing

In [1]:
#Imports
import pandas as pd
import numpy as np
import re
import io
from pathlib import Path

In [2]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving contacts.zip to contacts.zip


In [3]:
# files unzipping
!unzip contacts.zip

Archive:  contacts.zip
   creating: contacts/
  inflating: __MACOSX/._contacts     
  inflating: contacts/1515. copy     
  inflating: __MACOSX/contacts/._1515. copy  
  inflating: contacts/.DS_Store      
  inflating: __MACOSX/contacts/._.DS_Store  
  inflating: contacts/318. copy      
  inflating: __MACOSX/contacts/._318. copy  
  inflating: contacts/351. copy      
  inflating: __MACOSX/contacts/._351. copy  
  inflating: contacts/1462. copy     
  inflating: __MACOSX/contacts/._1462. copy  


In [ ]:
with open('./contacts/1462.copy', 'r',encoding='utf-8-sig') as file:
    data = file.read()

In [8]:
paths = Path('/content/contacts').glob('**/[!.]*')
msg = []
for path in paths:
    # because path is object not string
    path_in_str = str(path)
    # Do thing with the path
    print(path_in_str)
    with io.open(path, 'r') as file:
      msg.append(file.read())

/content/contacts/1462. copy
/content/contacts/318. copy
/content/contacts/351. copy
/content/contacts/1515. copy


In [26]:
#message processing
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            if not re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", line) and \
            not re.findall(r"(\.com|\.org|IMAP|ESMTP|(EDT)|(ICT))", line): 
              message += line.strip()
              email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val+' '
    return email

def parse_into_emails(messages):
  emails = [parse_raw_message(message) for message in messages]
  return {'body': map_to_list(emails, 'body'), 
          'to': map_to_list(emails, 'to'), 
          'from_': map_to_list(emails, 'from')
          }

def map_to_list(dct, key):
  tmp = []
  for item in dct:
    tmp.append(item[key])
  return tmp 

In [49]:
#test
res = parse_into_emails(msg)

In [53]:
res['body']

['Jeff My sister is a Vice President for JPMorgan Chase Bank  and she supervisesconstruction and management of new buildings in Texas  She sent me thefollowing note regarding Enron Energy Services handling of their utilitya national basis  including Texas   We are building a call center in SanAntonio  and the City of San Antonio municipal electric wants to chargeus a 50 000 deposit for new power because 11 out of the last 12 paymentsmade by Enron have been late  and late fees have been paid   I am now goingto do an analysis on all of my regions   I would like a local Enroncontact  if one exists   Any ideasWhom in Enron should I tell her to contactThanks Steve BaileyEnron Global EP  Inc 7136467865',
 'Hi  Kristin   I guess we got our wires crossed last Friday about lunch Sorry about that   He told me that you guys have rescheduled for tomorrowreservation at Carrabas on Kirby in your Dads name at that time   Please letme know if you have a conflict   Thanks  SRSPS  I also have you down f

In [28]:
email_df = pd.DataFrame(res)

In [35]:
email_df['body'].values

array(['Jeff,My sister is a Vice President for JPMorgan Chase Bank, and she supervisesconstruction and management of new buildings in Texas. She sent me thefollowing note regarding Enron Energy Services\' handling of their utilitya national basis, including Texas.  We are building a call center in SanAntonio, and the City of San Antonio (municipal electric) wants to chargeus a $50,000 deposit for new power because 11 out of the last 12 paymentsmade by Enron have been late, and late fees have been paid.  I am now goingto do an analysis on all of my regions.  I would like a local Enroncontact, if one exists.  Any ideas?"Whom in Enron should I tell her to contact?Thanks,Steve BaileyEnron Global E&P, Inc.713-646-7865',
       "Hi, Kristin.  I guess we got our wires crossed last Friday about lunch.Sorry about that.  He told me that you guys have rescheduled for tomorrowreservation at Carrabas on Kirby in your Dad's name at that time.  Please letme know if you have a conflict.  Thanks, SRSPS

In [ ]:
#text processing
import string
import nltk
#nltk.download('punkt')
#nltk.download('wordnet')
nltk.download('stopwords')
#from nltk.stem import WordNetLemmatizer
#wordnet_lemmatizer = WordNetLemmatizer()
#from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [52]:
import string
tmp = []
for item in res['body']:
  temp = item.translate(str.maketrans(',.', '  '))
  tmp.append(temp.translate(str.maketrans('', '', string.punctuation)))
res['body'] = tmp  

In [ ]:
#exlude stop words
stop_words = stopwords.words('english')
tokens = [w for w in tokens if not w in stop_words]